In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime
from scipy import stats
from scipy.stats import chi2_contingency, normaltest, skew, kurtosis

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")


In [ ]:
# Load the insurance dataset
df = pd.read_csv('../data/raw/insurance_data.csv')

print(f"📊 Dataset Shape: {df.shape}")
print(f"📅 Data Types: {df.dtypes.value_counts().to_dict()}")
print(f"💾 Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display first few rows
print("\n🔍 First 5 rows of the dataset:")
display(df.head())

print("\n📋 Dataset Information:")
df.info()


In [ ]:
# Data Quality Assessment
print("🔍 Missing Values Analysis:")
missing_data = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_summary = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percentage
}).sort_values('Missing Count', ascending=False)

print(missing_summary[missing_summary['Missing Count'] > 0])

if missing_summary['Missing Count'].sum() == 0:
    print("✅ No missing values found in the dataset!")

# Check for duplicate records
duplicates = df.duplicated().sum()
print(f"\n🔄 Duplicate Records: {duplicates}")

# Data type verification
print("\n📊 Data Types Verification:")
for col in df.columns:
    dtype = df[col].dtype
    unique_count = df[col].nunique()
    print(f"{col:20} | {str(dtype):12} | Unique Values: {unique_count}")


In [ ]:
# Descriptive Statistics for Numerical Variables
print("📈 Descriptive Statistics for Numerical Variables:")
numerical_cols = df.select_dtypes(include=[np.number]).columns
desc_stats = df[numerical_cols].describe()

# Add additional statistical measures
additional_stats = pd.DataFrame({
    'skewness': df[numerical_cols].skew(),
    'kurtosis': df[numerical_cols].kurtosis(),
    'variance': df[numerical_cols].var()
}).round(4)

# Combine descriptive statistics
full_stats = pd.concat([desc_stats.round(2), additional_stats.T], axis=0)
display(full_stats)


In [ ]:
# Calculate overall Loss Ratio
overall_loss_ratio = df['TotalClaims'].sum() / df['TotalPremium'].sum()
print(f"📊 Overall Portfolio Loss Ratio: {overall_loss_ratio:.4f} ({overall_loss_ratio*100:.2f}%)")

# Calculate Loss Ratio by different segments
print("\n🎯 Loss Ratio Analysis by Segments:")

# By Province
province_loss_ratio = df.groupby('Province').agg({
    'TotalClaims': 'sum',
    'TotalPremium': 'sum',
    'PolicyID': 'count'
}).reset_index()
province_loss_ratio['LossRatio'] = province_loss_ratio['TotalClaims'] / province_loss_ratio['TotalPremium']
province_loss_ratio = province_loss_ratio.sort_values('LossRatio', ascending=False)
province_loss_ratio.columns = ['Province', 'Total_Claims', 'Total_Premium', 'Policy_Count', 'Loss_Ratio']

print("\n📍 Loss Ratio by Province:")
display(province_loss_ratio.round(4))

# By Vehicle Type
vehicle_loss_ratio = df.groupby('VehicleType').agg({
    'TotalClaims': 'sum',
    'TotalPremium': 'sum',
    'PolicyID': 'count'
}).reset_index()
vehicle_loss_ratio['LossRatio'] = vehicle_loss_ratio['TotalClaims'] / vehicle_loss_ratio['TotalPremium']
vehicle_loss_ratio = vehicle_loss_ratio.sort_values('LossRatio', ascending=False)
vehicle_loss_ratio.columns = ['Vehicle_Type', 'Total_Claims', 'Total_Premium', 'Policy_Count', 'Loss_Ratio']

print("\n🚗 Loss Ratio by Vehicle Type:")
display(vehicle_loss_ratio.round(4))

# By Gender
gender_loss_ratio = df.groupby('Gender').agg({
    'TotalClaims': 'sum',
    'TotalPremium': 'sum',
    'PolicyID': 'count'
}).reset_index()
gender_loss_ratio['LossRatio'] = gender_loss_ratio['TotalClaims'] / gender_loss_ratio['TotalPremium']
gender_loss_ratio = gender_loss_ratio.sort_values('LossRatio', ascending=False)
gender_loss_ratio.columns = ['Gender', 'Total_Claims', 'Total_Premium', 'Policy_Count', 'Loss_Ratio']

print("\n👥 Loss Ratio by Gender:")
display(gender_loss_ratio.round(4))


In [ ]:
# Distribution Analysis of Key Financial Variables
financial_vars = ['TotalPremium', 'TotalClaims', 'CustomValueEstimate']

print("📈 Distribution Analysis of Financial Variables:")
for var in financial_vars:
    print(f"\n--- {var} ---")
    data = df[var]
    
    # Basic statistics
    print(f"Mean: ${data.mean():,.2f}")
    print(f"Median: ${data.median():,.2f}")
    print(f"Std Dev: ${data.std():,.2f}")
    print(f"Skewness: {skew(data):.4f}")
    print(f"Kurtosis: {kurtosis(data):.4f}")
    
    # Test for normality
    stat, p_value = normaltest(data)
    print(f"Normality Test (D'Agostino): p-value = {p_value:.6f}")
    if p_value < 0.05:
        print("❌ Data is NOT normally distributed")
    else:
        print("✅ Data appears normally distributed")

# Outlier Detection using IQR method
print("\n🎯 Outlier Detection (IQR Method):")
for var in financial_vars:
    data = df[var]
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    outlier_percentage = (len(outliers) / len(data)) * 100
    
    print(f"\n{var}:")
    print(f"  Lower Bound: ${lower_bound:,.2f}")
    print(f"  Upper Bound: ${upper_bound:,.2f}")
    print(f"  Outliers: {len(outliers)} ({outlier_percentage:.2f}%)")
    if len(outliers) > 0:
        print(f"  Max Outlier: ${outliers.max():,.2f}")
        print(f"  Min Outlier: ${outliers.min():,.2f}")


In [ ]:
# Temporal Analysis
print("📅 Temporal Trends Analysis:")

# Convert TransactionDate to datetime if not already
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['Year_Month'] = df['TransactionDate'].dt.to_period('M')

# Monthly aggregations
monthly_stats = df.groupby('Year_Month').agg({
    'TotalPremium': ['sum', 'mean', 'count'],
    'TotalClaims': ['sum', 'mean'],
    'HasClaim': ['sum', 'mean'],
    'CustomValueEstimate': 'mean'
}).round(2)

# Flatten column names
monthly_stats.columns = ['_'.join(col).strip() for col in monthly_stats.columns]
monthly_stats.reset_index(inplace=True)

# Calculate additional metrics
monthly_stats['Claim_Frequency'] = monthly_stats['HasClaim_sum'] / monthly_stats['TotalPremium_count'] * 100
monthly_stats['Avg_Claim_Severity'] = monthly_stats['TotalClaims_sum'] / monthly_stats['HasClaim_sum']
monthly_stats['Loss_Ratio'] = monthly_stats['TotalClaims_sum'] / monthly_stats['TotalPremium_sum']

# Handle division by zero
monthly_stats['Avg_Claim_Severity'] = monthly_stats['Avg_Claim_Severity'].fillna(0)
monthly_stats['Loss_Ratio'] = monthly_stats['Loss_Ratio'].fillna(0)

print("📈 Monthly Trends Summary:")
display(monthly_stats.round(2))

# Identify trends
print("\n🔍 Trend Analysis:")
print(f"Claim Frequency Range: {monthly_stats['Claim_Frequency'].min():.2f}% - {monthly_stats['Claim_Frequency'].max():.2f}%")
print(f"Average Claim Severity Range: ${monthly_stats['Avg_Claim_Severity'].min():,.2f} - ${monthly_stats['Avg_Claim_Severity'].max():,.2f}")
print(f"Loss Ratio Range: {monthly_stats['Loss_Ratio'].min():.4f} - {monthly_stats['Loss_Ratio'].max():.4f}")

# Correlation with time (using month number as proxy)
monthly_stats['Month_Number'] = range(1, len(monthly_stats) + 1)
freq_correlation = monthly_stats['Claim_Frequency'].corr(monthly_stats['Month_Number'])
severity_correlation = monthly_stats['Avg_Claim_Severity'].corr(monthly_stats['Month_Number'])

print(f"\n📊 Correlation with Time:")
print(f"Claim Frequency vs Time: {freq_correlation:.4f}")
print(f"Claim Severity vs Time: {severity_correlation:.4f}")

if abs(freq_correlation) > 0.3:
    trend_direction = "increasing" if freq_correlation > 0 else "decreasing"
    print(f"🔺 Notable {trend_direction} trend in claim frequency over time")
if abs(severity_correlation) > 0.3:
    trend_direction = "increasing" if severity_correlation > 0 else "decreasing"
    print(f"🔺 Notable {trend_direction} trend in claim severity over time")


In [ ]:
# Vehicle Analysis
print("🚗 Vehicle Analysis - Claims by Make and Type:")

# Analysis by Vehicle Make
vehicle_make_analysis = df.groupby('VehicleMake').agg({
    'TotalClaims': ['sum', 'mean', 'count'],
    'TotalPremium': ['sum', 'mean'],
    'HasClaim': ['sum', 'mean'],
    'CustomValueEstimate': 'mean'
}).round(2)

vehicle_make_analysis.columns = ['_'.join(col).strip() for col in vehicle_make_analysis.columns]
vehicle_make_analysis.reset_index(inplace=True)

# Calculate additional metrics
vehicle_make_analysis['Avg_Claim_Amount'] = vehicle_make_analysis['TotalClaims_sum'] / vehicle_make_analysis['HasClaim_sum']
vehicle_make_analysis['Claim_Frequency_Pct'] = vehicle_make_analysis['HasClaim_mean'] * 100
vehicle_make_analysis['Loss_Ratio'] = vehicle_make_analysis['TotalClaims_sum'] / vehicle_make_analysis['TotalPremium_sum']

# Handle division by zero
vehicle_make_analysis['Avg_Claim_Amount'] = vehicle_make_analysis['Avg_Claim_Amount'].fillna(0)

# Sort by different metrics
print("📊 Vehicle Makes Ranked by Average Claim Amount:")
top_claim_makes = vehicle_make_analysis.sort_values('Avg_Claim_Amount', ascending=False)
display(top_claim_makes[['VehicleMake', 'TotalClaims_count', 'Avg_Claim_Amount', 'Claim_Frequency_Pct', 'Loss_Ratio']].round(2))

print("\n🏆 Top 3 Highest Risk Vehicle Makes:")
for i, row in top_claim_makes.head(3).iterrows():
    print(f"{row['VehicleMake']}: Avg Claim ${row['Avg_Claim_Amount']:,.2f}, Frequency {row['Claim_Frequency_Pct']:.1f}%")

print("\n✅ Top 3 Lowest Risk Vehicle Makes:")
for i, row in top_claim_makes.tail(3).iterrows():
    if row['Avg_Claim_Amount'] > 0:  # Only include makes with actual claims
        print(f"{row['VehicleMake']}: Avg Claim ${row['Avg_Claim_Amount']:,.2f}, Frequency {row['Claim_Frequency_Pct']:.1f}%")

# Vehicle Type Analysis
print("\n\n🚙 Vehicle Types Analysis:")
vehicle_type_analysis = df.groupby('VehicleType').agg({
    'TotalClaims': ['sum', 'mean'],
    'HasClaim': ['sum', 'mean'],
    'TotalPremium': 'sum',
    'CustomValueEstimate': 'mean'
}).round(2)

vehicle_type_analysis.columns = ['_'.join(col).strip() for col in vehicle_type_analysis.columns]
vehicle_type_analysis.reset_index(inplace=True)
vehicle_type_analysis['Avg_Claim_Amount'] = vehicle_type_analysis['TotalClaims_sum'] / vehicle_type_analysis['HasClaim_sum']
vehicle_type_analysis['Claim_Frequency_Pct'] = vehicle_type_analysis['HasClaim_mean'] * 100
vehicle_type_analysis = vehicle_type_analysis.sort_values('Avg_Claim_Amount', ascending=False)

display(vehicle_type_analysis[['VehicleType', 'Avg_Claim_Amount', 'Claim_Frequency_Pct', 'CustomValueEstimate_mean']].round(2))


In [ ]:
# 🎨 VISUALIZATION 1: Interactive Loss Ratio Heatmap by Province and Vehicle Type
print("🎨 Creating Visualization 1: Interactive Loss Ratio Heatmap")

# Create pivot table for heatmap
loss_ratio_pivot = df.groupby(['Province', 'VehicleType']).agg({
    'TotalClaims': 'sum',
    'TotalPremium': 'sum'
}).reset_index()

loss_ratio_pivot['Loss_Ratio'] = loss_ratio_pivot['TotalClaims'] / loss_ratio_pivot['TotalPremium']
heatmap_data = loss_ratio_pivot.pivot(index='Province', columns='VehicleType', values='Loss_Ratio')

# Create interactive heatmap using Plotly
fig1 = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='RdYlBu_r',
    colorbar=dict(title="Loss Ratio"),
    hoverongaps=False,
    hovertemplate='Province: %{y}<br>Vehicle Type: %{x}<br>Loss Ratio: %{z:.4f}<extra></extra>'
))

fig1.update_layout(
    title={
        'text': '🔥 Loss Ratio Heatmap: Risk Patterns by Province & Vehicle Type',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18}
    },
    xaxis_title="Vehicle Type",
    yaxis_title="Province",
    font=dict(size=12),
    height=600,
    width=900
)

fig1.show()
print("✅ Heatmap created - Shows risk concentration patterns across geography and vehicle types")


In [ ]:
# 🎨 VISUALIZATION 2: Multi-Dimensional Bubble Chart - Risk vs Profitability
print("🎨 Creating Visualization 2: Risk vs Profitability Bubble Chart")

# Prepare data for bubble chart
bubble_data = df.groupby('VehicleMake').agg({
    'TotalClaims': 'sum',
    'TotalPremium': 'sum',
    'HasClaim': 'mean',
    'PolicyID': 'count',
    'CustomValueEstimate': 'mean'
}).reset_index()

bubble_data['Loss_Ratio'] = bubble_data['TotalClaims'] / bubble_data['TotalPremium']
bubble_data['Claim_Frequency'] = bubble_data['HasClaim'] * 100
bubble_data['Portfolio_Size'] = bubble_data['PolicyID']

# Create bubble chart
fig2 = px.scatter(
    bubble_data,
    x='Claim_Frequency',
    y='Loss_Ratio',
    size='Portfolio_Size',
    color='CustomValueEstimate',
    hover_name='VehicleMake',
    hover_data={
        'Claim_Frequency': ':.2f',
        'Loss_Ratio': ':.4f',
        'Portfolio_Size': ':,',
        'CustomValueEstimate': ':,.0f'
    },
    labels={
        'Claim_Frequency': 'Claim Frequency (%)',
        'Loss_Ratio': 'Loss Ratio',
        'Portfolio_Size': 'Number of Policies',
        'CustomValueEstimate': 'Avg Vehicle Value ($)'
    },
    title='💼 Risk vs Profitability: Vehicle Make Analysis<br><sub>Size = Portfolio Size, Color = Average Vehicle Value</sub>',
    color_continuous_scale='Viridis'
)

# Add quadrant lines
avg_freq = bubble_data['Claim_Frequency'].mean()
avg_loss = bubble_data['Loss_Ratio'].mean()

fig2.add_hline(y=avg_loss, line_dash="dash", line_color="red", 
               annotation_text="Average Loss Ratio", annotation_position="right")
fig2.add_vline(x=avg_freq, line_dash="dash", line_color="red",
               annotation_text="Average Claim Frequency", annotation_position="top")

fig2.update_layout(
    width=1000,
    height=700,
    title_font_size=16,
    title_x=0.5
)

fig2.show()
print("✅ Bubble chart created - Reveals risk-profitability relationships and portfolio concentration")


In [ ]:
# 🎨 VISUALIZATION 3: Temporal Evolution Dashboard with Multiple Metrics
print("🎨 Creating Visualization 3: Temporal Evolution Dashboard")

# Create subplots
fig3 = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Monthly Claim Frequency Trend', 'Monthly Loss Ratio Evolution',
                   'Premium vs Claims Volume', 'Risk Score by Vehicle Age'),
    specs=[[{"secondary_y": True}, {"secondary_y": False}],
           [{"secondary_y": True}, {"secondary_y": False}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# Convert Year_Month to string for plotting
monthly_stats['Month_Str'] = monthly_stats['Year_Month'].astype(str)

# Plot 1: Claim Frequency Trend with Volume
fig3.add_trace(
    go.Scatter(x=monthly_stats['Month_Str'], y=monthly_stats['Claim_Frequency'],
               mode='lines+markers', name='Claim Frequency (%)',
               line=dict(color='red', width=3)),
    row=1, col=1
)
fig3.add_trace(
    go.Bar(x=monthly_stats['Month_Str'], y=monthly_stats['TotalPremium_count'],
           name='Policy Count', opacity=0.6, marker_color='lightblue'),
    row=1, col=1, secondary_y=True
)

# Plot 2: Loss Ratio Evolution
fig3.add_trace(
    go.Scatter(x=monthly_stats['Month_Str'], y=monthly_stats['Loss_Ratio'],
               mode='lines+markers', name='Loss Ratio',
               line=dict(color='orange', width=3), fill='tonexty'),
    row=1, col=2
)

# Plot 3: Premium vs Claims Volume
fig3.add_trace(
    go.Scatter(x=monthly_stats['Month_Str'], y=monthly_stats['TotalPremium_sum'],
               mode='lines+markers', name='Premium Volume',
               line=dict(color='green', width=2)),
    row=2, col=1
)
fig3.add_trace(
    go.Scatter(x=monthly_stats['Month_Str'], y=monthly_stats['TotalClaims_sum'],
               mode='lines+markers', name='Claims Volume',
               line=dict(color='red', width=2)),
    row=2, col=1, secondary_y=True
)

# Plot 4: Risk Score by Vehicle Age
age_risk = df.groupby('VehicleAge').agg({
    'HasClaim': 'mean',
    'TotalClaims': 'mean'
}).reset_index()
age_risk['Risk_Score'] = age_risk['HasClaim'] * age_risk['TotalClaims']

fig3.add_trace(
    go.Bar(x=age_risk['VehicleAge'], y=age_risk['Risk_Score'],
           name='Risk Score', marker_color='purple'),
    row=2, col=2
)

# Update layout
fig3.update_layout(
    title_text="📊 Insurance Analytics Dashboard: Temporal & Risk Evolution",
    title_x=0.5,
    title_font_size=18,
    height=800,
    width=1200,
    showlegend=True
)

# Update y-axis labels
fig3.update_yaxes(title_text="Claim Frequency (%)", row=1, col=1)
fig3.update_yaxes(title_text="Policy Count", row=1, col=1, secondary_y=True)
fig3.update_yaxes(title_text="Loss Ratio", row=1, col=2)
fig3.update_yaxes(title_text="Premium Volume ($)", row=2, col=1)
fig3.update_yaxes(title_text="Claims Volume ($)", row=2, col=1, secondary_y=True)
fig3.update_yaxes(title_text="Risk Score", row=2, col=2)

# Update x-axis labels
fig3.update_xaxes(title_text="Month", row=2, col=1)
fig3.update_xaxes(title_text="Month", row=2, col=2)
fig3.update_xaxes(title_text="Vehicle Age (Years)", row=2, col=2)

fig3.show()
print("✅ Dashboard created - Comprehensive view of temporal trends and risk factors")
